In [1]:
import pandas as pd

df = pd.read_csv("data/train.csv", index_col=0)
df=df[~df['labels'].str.contains(":")]

## Sparse Matrix Encoding

In [2]:
import ast
from scipy.sparse import csr_matrix, vstack, save_npz, load_npz
import numpy as np
from sklearn.feature_extraction import DictVectorizer

X=[]
y=None

for i, r in df.iterrows():
    X.append(ast.literal_eval("{"+df.features[i].replace(" ",",")+"}"))
    y_i=(ast.literal_eval("["+df.labels[i]+"]"))
    y_i=csr_matrix((np.ones(len(y_i)), (np.zeros(len(y_i)), y_i)), shape=(1, NUM_LABELS),dtype=int)
    if y==None:
        y=y_i
    else:
        y=vstack((y,y_i))

v = DictVectorizer()
X=v.fit_transform(X)


NameError: name 'NUM_LABELS' is not defined

In [ ]:
save_npz("data/X_sparse.npz",X)
save_npz("data/y_sparse.npz",y)


In [ ]:
from scipy.sparse import load_npz

X=load_npz("data/X_sparse.npz")
y=load_npz("data/y_sparse.npz")


In [ ]:
#downsample for testing purposes
from sklearn.utils import resample
pct=50
X, y = resample(X,y,n_samples=int(round(pct*y.shape[0]/100)))

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
                                X, y, test_size=0.25, random_state=42)


In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier #O(n_classes^2)

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.metrics import label_ranking_average_precision_score


In [ ]:
ovrnb = OneVsRestClassifier(MultinomialNB(),n_jobs=16).fit(X_train, y_train)
y_train_hat=ovrnb.predict(X_train)
print(label_ranking_average_precision_score(y_train.todense(),y_train_hat.todense()))
y_val_hat=ovrnb.predict(X_val)
print(label_ranking_average_precision_score(y_val.todense(),y_val_hat.todense()))


In [ ]:
ovrsvm = OneVsRestClassifier(LinearSVC(       ),n_jobs=16).fit(X_train, y_train)
y_train_hat=ovrsvm.predict(X_train)
print(label_ranking_average_precision_score(y_train.todense(),y_train_hat.todense()))
y_val_hat=ovrsvm.predict(X_val)
print(label_ranking_average_precision_score(y_val.todense(),y_val_hat.todense()))


In [ ]:
#from sklearn.model_selection import cross_validate
#cv_results = cross_validate(clf, X_train, y_train)
#cv_results

In [ ]:
#Cs=[.001,.002,.005,.01,.02,.05,.1,.2,.5,1,2,5,10,12,15,100]
#Cs=[.001,.01,.1,1,10,100]
Cs=[.02,.05,.1,.2,.5,1]
Cs=np.logspace(-1.3, 0, 6)

print(Cs)

for C in Cs:
    print("{:.3f}".format(C))
    ovrsvm = OneVsRestClassifier(LinearSVC(C=C),n_jobs=16).fit(X_train, y_train)
    y_val_hat=ovrsvm.predict(X_val)
    print("\t{:.4f}".format(label_ranking_average_precision_score(y_val.todense(),y_val_hat.todense())))
